In [7]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter

from market_dynamics import bs_delta, bs_generator, bs_call_price
from data import SimulationData
from utils import call_payoff, stochastic_integral
from models import ControlNet
from train import train, test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
comment = ""
log_dir = os.path.join('experiments', current_time + '_' + comment)
writer = SummaryWriter(log_dir)

In [8]:
n_simulations = 500 
initial_value = 100
sigma = 0.2
n_steps = 50
rf = 0
strike = 95
fc_dims = [16, 16]
LR = 0.01
EPOCHS = 20
p_train = 0.9

bs_params = {"n_simulations": n_simulations,
             "n_steps": n_steps,
             "initial_value": initial_value,
             "sigma": sigma,
             }

price_params = {"n_steps": n_steps,
                "initial_value": initial_value,
                "sigma": sigma,
                "rf": rf,
                "strike": strike,
                }

payoff_params = {"strike": strike}

n_train = int(p_train * n_simulations)
n_test = n_simulations - n_train

dt = SimulationData(bs_generator, bs_params, bs_call_price, price_params, call_payoff, payoff_params)
dt_train, dt_test = torch.utils.data.random_split(dt, [n_train, n_test])
data_loader_train = DataLoader(dt_train, batch_size=20, shuffle=True)
cn = ControlNet(n_steps, 1, fc_dims, 1)

optimizer = Adam(cn.parameters(), lr=LR)
criterion = torch.nn.MSELoss()
train(data_loader_train, cn, criterion , optimizer, EPOCHS, writer)

Epoch 0: : 23batch [00:00, 26.92batch/s, loss=10.6]
Epoch 1: : 23batch [00:00, 26.26batch/s, loss=1.95]
Epoch 2: : 23batch [00:00, 26.84batch/s, loss=3.2]
Epoch 3: : 23batch [00:00, 25.70batch/s, loss=2.1]
Epoch 4: : 23batch [00:00, 29.07batch/s, loss=1.85]
Epoch 5: : 23batch [00:00, 26.22batch/s, loss=1.59]
Epoch 6: : 23batch [00:00, 33.60batch/s, loss=1.25]
Epoch 7: : 23batch [00:00, 33.60batch/s, loss=1.73]
Epoch 8: : 23batch [00:00, 36.71batch/s, loss=3.95]
Epoch 9: : 23batch [00:00, 32.85batch/s, loss=2.6]
Epoch 10: : 23batch [00:00, 31.21batch/s, loss=1.78]
Epoch 11: : 23batch [00:00, 35.05batch/s, loss=2.49]
Epoch 12: : 23batch [00:00, 29.24batch/s, loss=2.02]
Epoch 13: : 23batch [00:00, 28.90batch/s, loss=1.43]
Epoch 14: : 23batch [00:00, 33.25batch/s, loss=1.6]
Epoch 15: : 23batch [00:00, 26.92batch/s, loss=1.19]
Epoch 16: : 23batch [00:00, 27.29batch/s, loss=2.26]
Epoch 17: : 23batch [00:01, 13.14batch/s, loss=1.58]
Epoch 18: : 23batch [00:02, 10.73batch/s, loss=1.81]
Epoch 1

In [4]:
def delta(x):
    return bs_delta(n_steps, x, sigma, rf, strike)

In [5]:
# Evaluation In-Sample

_ ,model_loss = test(data_loader_train, cn, criterion)

l = len(data_loader_train.dataset)
x, x_inc, payoff, price = data_loader_train.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.5849, grad_fn=<MseLossBackward>)
tensor(2.8651)


In [6]:
# Evaluation Out-Sample
data_loader_test = DataLoader(dt_test, batch_size=20, shuffle=True)

_ ,model_loss = test(data_loader_test, cn, criterion)

l = len(data_loader_test.dataset)
x, x_inc, payoff, price = data_loader_test.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.9028, grad_fn=<MseLossBackward>)
tensor(2.7848)
